In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 8.4MB/s 


In [0]:
# https://github.com/spro/char-rnn.pytorch

import unidecode
import string
import random
import time
import math
import torch
from torch.autograd import Variable

USE_CUDA = True

# Reading and un-unicode-encoding data

all_characters = string.printable
n_characters = len(all_characters)
SOS = n_characters
EOS = n_characters + 1
n_characters += 2

def read_file(filename):
    file = unidecode.unidecode(open(filename).read())
    return file, len(file)

# Turning a string into a tensor

def char_tensor(string):
    size = len(string) + 1
    tensor = torch.zeros(size).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    tensor[-1] = EOS
    tensor = Variable(tensor)
    if USE_CUDA:
        tensor = tensor.cuda()
    return tensor

# Turn a tensor into a string

def index_to_char(top_i):
    if top_i == EOS:
        return '$'
    elif top_i == SOS:
        return '^'
    else:
        return all_characters[top_i]

def tensor_to_string(t):
    s = ''
    for i in range(t.size(0)):
        ti = t[i]
        top_k = ti.data.topk(1)
        top_i = top_k[1][0]
        s += index_to_char(top_i)
        if top_i == EOS: break
    return s

def longtensor_to_string(t):
    s = ''
    for i in range(t.size(0)):
        top_i = t.data[i]
        s += index_to_char(top_i)
    return s

# Readable time elapsed

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)



#MODEL

In [0]:

#https://github.com/spro/pytorch-text-vae
import torch
import torch.nn as nn
from torch.autograd import variable
import torch.nn.functional as F

MIN_LENGTH = 10
MAX_LENGTH = 50
MAX_SAMPLE = False
MAX_SAMPLE = True

In [0]:
class Encoder(nn.Module):
  
    def sample(self, mu, logvar):
      
        eps = Variable(torch.randn(mu.size()))
        
        if USE_CUDA:
          
            eps = eps.cuda()
         
        std = torch.exp(logvar / 2.0)
        
        return mu + eps * std
    
      

In [0]:
class EncoderRNN(Encoder):
    
    def __init__(self, input_size, hidden_size, output_size, n_layers = 1, bidirectional = True):
        
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        
        self.hidden_size = hidden_size
        
        self.output_size = output_size
        
        self.n_layers = n_layers
        
        self.bidirectional = bidirectional
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout = 0.1, bidirectional = bidirectional)
        self.o2p = nn.Linear(hidden_size, output_size*2)
        
    
    def forward(self, input):
        embedded = self.embed(input).unsqueeze(1)

        output, hidden = self.gru(embedded, None)
        output = output[-1] # Take only the last value
        if self.bidirectional:
            output = output[:, :self.hidden_size] + output[: ,self.hidden_size:] # Sum bidirectional outputs

        ps = self.o2p(output)
        mu, logvar = torch.chunk(ps, 2, dim=1)
        z = self.sample(mu, logvar)
        
        return mu, logvar, z

In [0]:
class DecoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, n_layers = 1, dropout_p = 0.1):
        super(DecoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.embed = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.z2h = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size + input_size, hidden_size, n_layers, dropout = dropout_p)
        self.i2h = nn.Linear(hidden_size + input_size, hidden_size)
        self.h2o = nn.Linear(hidden_size*2, hidden_size)
        self.out = nn.Linear(hidden_size + input_size, output_size)
        
        
    def sample(self, output, temperature):
        if MAX_SAMPLE:
            
            top_i = output.data.topk(1)[1][0][0]
        
        else:
            
            output_dist = output.data.view(-1).div(temperature).exp()
            top_i = torch.multinomial(output_dist, 1)[0]
        
        input = Variable(torch.LongTensor([top_i]))
        if USE_CUDA:
            input = input.cuda()
        
        return input, top_i
    
    def forward(self, z, inputs, temperature):
        n_steps = inputs.size(0)
        outputs = Variable(torch.zeros(n_steps, 1, self.output_size))
        if USE_CUDA:
            outputs = outputs.cuda()
        
        input = Variable(torch.LongTensor([SOS]))
        if USE_CUDA:
            input = input.cuda()
        
        hidden = self.z2h(z).unsqueeze(0).repeat(self.n_layers, 1, 1)
        
        for i in range(n_steps):
            output, hidden = self.step(i, z, input, hidden, temperature)
            outputs[i] = output
            
            use_teacher_forcing = random.random() < temperature
            
            if use_teacher_forcing:
                input = inputs[i]
            
            else:
                input, top_i = self.sample(output, temperature)
            
            
        
        
        return outputs.squeeze(1)
     
    def generate(self, z, n_steps, temperature):
        outputs = Variable(torch.zeros(n_steps, 1, self.output_size))
        
        if USE_CUDA:
            ouputs = outputs.cuda()
        
        input = Variable(torch.LongTensor([SOS]))
        
        if USE_CUDA:
            input = input.cuda()
        hidden = self.z2h(z).unsqueeze(0).repeat(self.n_layers, 1, 1)
        
        for i in range(n_steps):
            output, hidden = self.step(i, z, input, hidden, temperature)
            outputs[i] = output
            input, top_i = self.sample(output, temperature)
            if top_i == EOS: break
        
        return outputs.squeeze(1)
   
  
  
  
    def step(self, s, z, input, hidden, temperature = 0.3):
        
        input = F.relu(self.embed(input))
        input = input.squeeze(0) #added manually 
        input = input.unsqueeze(0) #added manually
#         print(input.size())
#         print('Hi')
#         print(z.size())
        input = torch.cat((input, z), 1)
        input = input.unsqueeze(0)
        output, hidden = self.gru(input, hidden)
        output = output.squeeze(0)
        output = torch.cat((output, z), 1)
        output = self.out(output)
        return output, hidden
        
        
        
        
        

In [0]:
class VAE(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
   
    def forward(self, inputs, temperature = 1.0):
        m, l, z = self.encoder(inputs)
        decoded = self.decoder(z, inputs, temperature)
        return m, l, z, decoded
      

###Testing the model

In [9]:
    hidden_size = 20
    embed_size = 10
    e = EncoderRNN(n_characters, hidden_size, embed_size)
    d = DecoderRNN(embed_size, hidden_size, n_characters, 2)
    if USE_CUDA:
        e.cuda()
        d.cuda()
    vae = VAE(e, d)
    m, l, z, decoded = vae(char_tensor('@spro'))
    print('m =', m.size())
    print('l =', l.size())
    print('z =', z.size())
    print('decoded', tensor_to_string(decoded))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


m = torch.Size([1, 10])
l = torch.Size([1, 10])
z = torch.Size([1, 10])
decoded HHHHll


#Train

In [10]:
!pip install sconce
import sconce 

In [0]:
import sconce
import sys


hidden_size = 500
embed_size = 50
learning_rate = 0.0001
n_epochs = 1000
grad_clip = 1.0

kld_start_inc = 10000
kld_weight = 0.05
kld_max = 0.1
kld_inc = 0.000002
temperature = 0.9
temperature_min = 0.5
temperature_dec = 0.000002

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
file, file_len = read_file('drive/My Drive/Data/beatles.txt')

print(file_len)


70685


In [21]:
lines = [line.strip() for line in file.split('\n')]
print(len(lines))

1846


In [0]:
def good_size(line):
    return len(line) >= MIN_LENGTH and len(line) <= MAX_LENGTH

def good_content(line):
    return 'http' not in line and '/' not in line

In [0]:
lines = [line for line in lines if good_size(line) and good_content(line)]

In [24]:
print('n lines', len(lines))

n lines 1462


In [0]:
def random_training_set():
  line = random.choice(lines)
  inp  = char_tensor(line)
  target = char_tensor(line)
  return inp, target

In [26]:
e = EncoderRNN(n_characters, hidden_size, embed_size)
d = DecoderRNN(embed_size, hidden_size, n_characters, 2)
vae = VAE(e, d)
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()

if USE_CUDA:
    vae.cuda()
    criterion.cuda()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
log_every = 20
save_every = 500


def save():
    save_filename = 'vae.pt'
    torch.save(vae, save_filename)
    print('Saved as %s' % save_filename)


In [0]:
 for epoch in range(n_epochs):
        input, target = random_training_set()

        optimizer.zero_grad()

        m, l, z, decoded = vae(input, temperature)
        if temperature > temperature_min:
            temperature -= temperature_dec

        loss = criterion(decoded, target)
       # job.record(epoch, loss.data[0])

        KLD = (-0.5 * torch.sum(l - torch.pow(m, 2) - torch.exp(l) + 1, 1)).mean().squeeze()
        loss += KLD * kld_weight

        if epoch > kld_start_inc and kld_weight < kld_max:
            kld_weight += kld_inc

        loss.backward()
        # print('from', next(vae.parameters()).grad.data[0][0])
        ec = torch.nn.utils.clip_grad_norm(vae.parameters(), grad_clip)
        # print('to  ', next(vae.parameters()).grad.data[0][0])
        optimizer.step()

        if epoch % log_every == 0:
            print('[%d] %.4f (k=%.4f, t=%.4f, kl=%.4f, ec=%.4f)' % (
                epoch, loss.item(), kld_weight, temperature, KLD.item(), ec
            ))
            print('   (target) "%s"' % longtensor_to_string(target))
            generated = vae.decoder.generate(z, MAX_LENGTH, temperature)
            print('(generated) "%s"' % tensor_to_string(generated))
            print('')

        if epoch > 0 and epoch % save_every == 0:
            save()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


[0] 2.7140 (k=0.0500, t=0.8995, kl=0.0972, ec=4.1992)
   (target) "They are the eggmen$"
(generated) "oe oe oo oe ae ae ae ae ae ae ae to ae ae ae ae ae"

[20] 2.9336 (k=0.0500, t=0.8994, kl=0.1252, ec=3.2758)
   (target) "By stealing the girl of his fancy$"
(generated) "e to to to to to to to ton ton to to to to ton to "

[40] 3.1742 (k=0.0500, t=0.8994, kl=0.2801, ec=4.1061)
   (target) "Is keeping perfectly still$"
(generated) "oo to to to to to to to to to to to to to to to to"

[60] 2.4782 (k=0.0500, t=0.8994, kl=0.1570, ec=2.7949)
   (target) "All the money's gone, nowhere to go$"
(generated) "e to to to to to toe to to to to to to to to to to"

[80] 2.8044 (k=0.0500, t=0.8993, kl=0.1785, ec=3.6821)
   (target) "Imagine all the people$"
(generated) "oo to to to to to to to to so to to to so to so to"

[100] 2.6834 (k=0.0500, t=0.8993, kl=0.0983, ec=2.6903)
   (target) "And I do appreciate you being round$"
(generated) "ou to to to to to to to to to to to to to to to to"

[120] 2.

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


[520] 2.2113 (k=0.0500, t=0.8984, kl=0.0842, ec=2.8557)
   (target) "And you're working for no one but me.$"
(generated) "All tou she love sand and and and and and and and "

[540] 2.6660 (k=0.0500, t=0.8984, kl=0.0534, ec=4.5921)
   (target) "Everywhere people stare$"
(generated) "I the the the the the the the the the the the the "

[560] 2.5044 (k=0.0500, t=0.8984, kl=0.1557, ec=3.4490)
   (target) "Eight days a week$"
(generated) "All the the the say the say the say the say$"

[580] 2.7772 (k=0.0500, t=0.8983, kl=0.0983, ec=3.0831)
   (target) "While my guitar gently weeps$"
(generated) "All tou tou tou sou tou sou sou tou sou sou sand a"

[600] 2.6684 (k=0.0500, t=0.8983, kl=0.0945, ec=3.4457)
   (target) "Has left a pool of tears$"
(generated) "All the the san the san the san$"

[620] 2.0681 (k=0.0500, t=0.8982, kl=0.1520, ec=2.2813)
   (target) "Well, you can indicate everything you see$"
(generated) "And the the the the the the the the the the the th"

[640] 2.1332 (k=0.0500, t=